# Imports

In [ ]:
%%html
<style>
.cm-s-ipython .CodeMirror-matchingbracket { color: LimeGreen !important;}
</style>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import pprint
from scipy import stats
import random
from datetime import datetime
import json
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from pprint import pprint
import time
from pandas import read_csv
from datetime import datetime
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.utils import np_utils
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from keras.wrappers.scikit_learn import KerasRegressor,KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.optimizers import SGD
from sklearn import metrics
import keras
import json
from sklearn.model_selection import ParameterGrid
from xgboost.sklearn import XGBClassifier

In [ ]:
from sklearn import preprocessing

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Functions

In [ ]:
def pre_process_df(df,verbose = 0):
    
    if verbose==1: print("Columns before : {}".format(df.shape[1]))
    
    df['time_period'] = df['time_hours']//8 + 1
    
    df = build_pseudo_mods(df)
    
    #Drop unwanted columns
    df_dropped = df.drop(columns=['Unnamed: 0','stash_id','stash_feed','item_category','_id','date','time_hours',
                                  'date_month','date_year','item_name','league','rarity','time','socket_colors',
                                  'price_currency','price_raw','date_day','time_minutes'],errors='ignore')
    
    if verbose==1: print("Columns after base drop : {}".format(df_dropped.shape[1]))
        
    # delete columns with no values in them, which means only zero
    proc_df = df_dropped.loc[:,(df!=0).any (axis=0)]
        
    if verbose==1: pprint("Removed columns with no values in them : {}".format(df_dropped.shape[1]-proc_df.shape[1]))
    
    # Clean up of data. 
    ## Prices were imported as 'Object' not 'float'. We need to convert them to float.
    proc_df[['price_amount']] = pd.to_numeric(proc_df['price_amount'],errors='coerce')     
    
    # Remove rows where price_amount is NaN
    proc_df = proc_df[pd.notnull(proc_df['price_amount'])]
    proc_df = proc_df.loc[proc_df['price_amount']>0]
    
    return proc_df

def number_of_resistances(x):
    number_of_resistances = 0
    single_resistance_mods = ['ex_#% to chaos resistance','ex_#% to cold resistance','ex_#% to fire resistance','ex_#% to lightning resistance']
    triple_elemental_resistance_mods = ['co_#% to all elemental resistances','im_#% to all elemental resistances']
    elemental_resistance_mods = ['co_#% to all elemental resistances','ex_#% to cold resistance','ex_#% to fire resistance','ex_#% to lightning resistance','im_#% to all elemental resistances']
    
    for mod in triple_elemental_resistance_mods:
        if mod in x and x[mod]>0 : number_of_resistances = 3
    for mod in single_resistance_mods:
        if mod in x and x[mod]>0 : number_of_resistances = number_of_resistances+1
    return number_of_resistances

def number_of_ele_resistances(x):
    number_of_ele_resistances = 0
    single_resistance_mods = ['ex_#% to chaos resistance','ex_#% to cold resistance','ex_#% to fire resistance','ex_#% to lightning resistance']
    triple_elemental_resistance_mods = ['co_#% to all elemental resistances','im_#% to all elemental resistances']
    elemental_resistance_mods = ['co_#% to all elemental resistances','ex_#% to cold resistance','ex_#% to fire resistance','ex_#% to lightning resistance','im_#% to all elemental resistances']
    single_ele_resistance_mods = ['ex_#% to cold resistance','ex_#% to fire resistance','ex_#% to lightning resistance']
    
    for mod in triple_elemental_resistance_mods:
        if mod in x and x[mod]>0 : return 3
    for mod in single_ele_resistance_mods:
        if mod in x and x[mod]>0 : number_of_ele_resistances = number_of_ele_resistances+1
    return number_of_ele_resistances

def total_elemental_resistance(x):
    total_ele_res = 0
    single_resistance_mods = ['ex_#% to chaos resistance','ex_#% to cold resistance','ex_#% to fire resistance','ex_#% to lightning resistance']
    triple_elemental_resistance_mods = ['co_#% to all elemental resistances','im_#% to all elemental resistances']
    elemental_resistance_mods = ['co_#% to all elemental resistances','ex_#% to cold resistance','ex_#% to fire resistance','ex_#% to lightning resistance','im_#% to all elemental resistances']
    single_ele_resistance_mods = ['ex_#% to cold resistance','ex_#% to fire resistance','ex_#% to lightning resistance']
    
    for mod in single_ele_resistance_mods:
        if mod in x and x[mod]>0 : total_ele_res = total_ele_res + x[mod]
    for mod in triple_elemental_resistance_mods:
        if mod in x and x[mod]>0 : total_ele_res = total_ele_res + (3 * x[mod])
    return total_ele_res

def total_resistance(x):
    total_res = 0
    single_resistance_mods = ['ex_#% to chaos resistance','ex_#% to cold resistance','ex_#% to fire resistance','ex_#% to lightning resistance']
    triple_elemental_resistance_mods = ['co_#% to all elemental resistances','im_#% to all elemental resistances']
    elemental_resistance_mods = ['co_#% to all elemental resistances','ex_#% to cold resistance','ex_#% to fire resistance','ex_#% to lightning resistance','im_#% to all elemental resistances']
    single_ele_resistance_mods = ['ex_#% to cold resistance','ex_#% to fire resistance','ex_#% to lightning resistance']
    
    for mod in single_resistance_mods:
        if mod in x and x[mod]>0 : total_res = total_res + x[mod]
    for mod in triple_elemental_resistance_mods:
        if mod in x and x[mod]>0 : total_res = total_res + (3 * x[mod])
    return total_res

def build_pseudo_mods(df):
    df['#_of_resistances'] = df.apply(lambda x : number_of_resistances(x),axis=1)
    df['#_of_ele_resistances'] = df.apply(lambda x : number_of_ele_resistances(x),axis=1)
    df['total_resistance'] = df.apply(lambda x : total_resistance(x),axis=1)
    df['total_elemental_resistance'] = df.apply(lambda x : total_elemental_resistance(x),axis=1)
                    
    return df
                    
def split_df_to_unique_item_names(df):

    unique_item_names = df['item_name'].value_counts(ascending=False)

    ascending_dataframes_per_name = {}

    for item_name in unique_item_names.index:
        dataF = df.loc[(df['item_name']== item_name)]
        if dataF.empty: continue
        if item_name not in ascending_dataframes_per_name:    
            ascending_dataframes_per_name[item_name] = dataF

    for item_name in ascending_dataframes_per_name:
        dataF = ascending_dataframes_per_name[item_name]
        dataF = dataF.loc[:,(dataF!=0).any(axis=0)]
        mask = dataF['price_amount'].notna()
        dataF = dataF[mask]
        ascending_dataframes_per_name[item_name] = dataF.reset_index()
        
    return ascending_dataframes_per_name

def compute_corr(df,method='spearman',filename=''):
    
    min_periods = int(len(df))*0.1
    cols = list(df.filter(regex='(Attacks per Second|Energy Shield|Elemental Damage|Critical Strike Chance|Physical Damage|influence|Armour|sockets_number|linked_sockets|Evasion Rating)|(?=^co_|ex_|im_|en_$)(^.*$)').columns.values)
    df[cols] = df[cols].replace({0:np.nan, 0.0:np.nan})
    #df[df.filter(regex='(?=^co_|ex_|im_|en_$)(^.*$)') <= 0.0] = np.nan
    corr = df.corr(method,min_periods = min_periods)
    corr = corr.dropna('columns',how='all')
    corr = corr.dropna('rows',how='all')
    
    return corr

def remove_outliers_IQR(item_dataframe,column_label = 'price_amount',high_quantile=0.75):
    '''Function removes outliers from a dataframe along the price_amount column by default.

    Input:
        df: pandas DataFrame
        column_label: along which column to check for outliers(default = 'price_amount')
        high_quantile: high-end quantile to use on boxplot'''
    
    Q1 = item_dataframe[column_label].quantile(1-high_quantile)
    Q3 = item_dataframe[column_label].quantile(high_quantile)
    IQR = Q3 - Q1
    new_df = item_dataframe[~((item_dataframe[column_label] < (Q1 - 1.5 * IQR))|(item_dataframe[column_label] > (Q3 + 1.5 * IQR)))]
    
    return new_df

def remove_outliers_zscore(item_dataframe,column_labels = ['price_amount'],threshold=3,show_results=False):
    '''Function removes outliers using z-score from a dataframe along the price_amount column by default.

    Input:
        df: pandas DataFrame
        column_label: along which columns to check for outliers(default = ['price_amount'])
        show_results: show results before and after removing outliers(default = False)
        size: vertical and horizontal size of the plot'''
    
    z_score = np.abs(stats.zscore(item_dataframe[column_labels]))
    new_df = item_dataframe[(z_score < threshold)]
    if show_results:
        data_outliers_index = np.where(z_score > threshold)[0]
        print('Data outliers for "{}":'.format(item_dataframe['item_name'][0]))
        for id in data_outliers_index:
              print('index: {:<10d}{}: {:<10f}'.format(id,column_labels[0],item_dataframe.iloc[id][column_labels[0]]))
        print('Removed {} rows'.format(item_dataframe.shape[0]-new_df.shape[0]))
    return new_df


def produce_decision_dataframe(item_df,correlations_df=pd.DataFrame,incl_outliers=True,method='z-score',threshold=2,quantile=0.8):
    
    d_df = item_df
    
    if not incl_outliers:
        if method == 'z-score' : 
            d_df = remove_outliers_zscore(d_df,threshold=threshold)
        elif method == 'IQR' :
            d_df = remove_outliers_IQR(d_df,high_quantile=quantile)
        else:
            raise Exception('\t\tWrong outlier mode. Valid options mode = [z-score | IQR]')
    
    if correlations_df.empty:
        columns = ['item_name','feature','corr_value','no_features','transactions','st_div','variance']
        correlations_df =  pd.DataFrame(columns=columns)
    
    corr=compute_corr(d_df,method='kendall')
    #corr_filtered = corr['price_amount'].filter(regex='(item_category|corrupted|Attacks per Second|Energy Shield|Elemental Damage|Critical Strike Chance|Physical Damage|influence|Armour|sockets_number|linked_sockets|Quality|Evasion Rating)|(?=^co_|ex_|im_|en_$)(^.*$)').drop(labels=['ex_conv_rate'],axis=0).dropna()
    corr_filtered = corr['price_amount'].filter(regex='(date_day|item_category|corrupted|Attacks per Second|Energy Shield|Elemental Damage|Critical Strike Chance|Physical Damage|influence|Armour|sockets_number|linked_sockets|Quality|Evasion Rating)|(?=^co_|ex_|im_|en_$)(^.*$)').dropna()
    for row in corr_filtered.index:
        correlations_df = correlations_df.append({'item_name':d_df['item_name'].unique()[0],
                                'feature':row,
                                'corr_value': corr_filtered[row],
                                'no_features':len(corr_filtered),
                                'transactions':d_df.groupby('item_name')['item_name'].count().values[0],
                                'st_div':d_df['price_amount'].describe()['std'],
                                'variance':d_df[['price_amount']].var(axis=0)},ignore_index=True)
    
    return correlations_df

def produce_corr_based_df(df_per_item_name,method='z-score',threshold=2,quantile=0.8):
    
    columns = ['item_name','feature','corr_value','no_features','transactions','st_div','variance']

    df =  pd.DataFrame(columns=columns)

    count = 0
    for dataF in df_per_item_name:
        count= count+1
        if count%200==0:
            print("Processed {} item_names".format(count))
        item_df = df_per_item_name[dataF]
        df = produce_decision_dataframe(item_df,df,incl_outliers=False,method=method,threshold=threshold,quantile=quantile)
        
    return df

def filter_decision_df(df, days=7, min_corr=0.1, min_no_features=2, min_std=5.0):
    
    min_trx = days*24
    
    df_filtered = df[(abs(df['corr_value'])>=min_corr) & \
                                          (df['transactions'] > min_trx) & \
                                          (df['st_div'] > min_std)]
    
    df_filtered['no_features'] = df_filtered.groupby('item_name')['item_name'].transform('count')
    df_filtered = df_filtered[df_filtered['no_features'] >= min_no_features]
    
    return df_filtered

def convert_column_values_string_to_rankInt(df) -> pd.DataFrame:
    for column in df.columns:
        if df[column].dtype == type(object):
            le = preprocessing.LabelEncoder()
            df[column] = le.fit_transform(df[column])
    return df

def flatten_column(df_column,method='median',round_base=2):
    if method=='median':
        return df_column.median()
    elif method=='mean':
        return round(df_column.mean(),round_base)
    
def interpolate_df(df,config):
    feature_series = []
    index = []
    item_features = df.columns
    for feature in item_features:
        if feature in config['features']:
            if config['features'][feature]=='median':
                inter_series_f = flatten_column(df[feature],method='median')
            elif config['features'][feature]=='mean':
                inter_series_f = flatten_column(df[feature],method='mean')
            else:
                inter_series_f = flatten_column(df[feature],method=config['default_flatten'])
        else:
            inter_series_f = flatten_column(df[feature],method=config['default_flatten'])
        
        index.append(feature)
        feature_series.append(inter_series_f)
    return pd.Series(feature_series,index)

def fill_and_plot(df,method='default',order=3):
    if method in ['spline','polynomial']:
        df_inter = df.interpolate(method=method,order=order)
    else:
        df_inter = df.interpolate(method=method)
    plt.figure(figsize=(20,10))
    df_inter['price_amount'].plot()
    plt.legend([method])
    return df_inter


def fill_time_periods(df,method = 'pchip',order=3):

    if method in ['spline','polynomial']:
        df = df.interpolate(method=method,order=order)
    else:
        df = df.interpolate(method=method)
    return df

def feature_selection(df, method="decision_tree",verbose=0,importance_threshold=0.15,max_no_of_features = 5):
    important_features = []
    print("Max number of features = {}".format(max_no_of_features))
    train = df.copy()
    new_df = df.drop(['price_amount','time','date','socket_colors','time_hours','item_category'],axis=1,errors='ignore')
    
    if method=='custom':
        df = new_df.copy()
        df_missing = df!=0
        df_missing_perc = df_missing.sum()/df.shape[0]
        for value in df_missing_perc[df_missing_perc>=0.001].index.values:
            important_features.append(value)
        important_features.append('price_amount')
        return important_features
    elif method=='decision_tree':
        model = RandomForestRegressor(random_state=20,max_depth=300)
        new_df = pd.get_dummies(new_df)
        model.fit(new_df,train.price_amount)
        features = new_df.columns
        importances = model.feature_importances_
        indices = np.argsort(importances)[-max_no_of_features:]  # top 10 features
        if verbose:
            for i in indices:
                print("Feature : {:40} --->importance [{}]".format(features[i][:40],importances[i].round(3)))
                      
        for i in indices:
            if importances[i] > importance_threshold:
                important_features.append(features[i])
        important_features.append('price_amount')
        return important_features
    
    elif method=='rfe':
        lreg = DecisionTreeRegressor()
        rfe = RFE(lreg, max_no_of_features-1)
        cols = new_df.columns
        rfe = rfe.fit(new_df, train.price_amount)
        sorted_ranking = sorted(zip(map(lambda x:round(x,5),rfe.ranking_),cols))
        
        for i in range(0,len(sorted_ranking)):
            if verbose==1: print("Feature : {:40} has weight [{}]".format(sorted_ranking[i][1][:35],sorted_ranking[i][0]))
#             if (sorted_ranking[i][0] <= max_no_of_features*0.3 and (len(important_features)<(max_no_of_features))):    
            if (len(important_features)<(max_no_of_features-1)):
                important_features.append(sorted_ranking[i][1])
                
        important_features.append('price_amount')
        return important_features
    
def dynamic_bins(df_bins, no_rows, min_percent=3, step=0.5, max_cum_step=50):
    count = 0
    left = 0
    percent = 0
    step_value = 0
    bins = [0]
    labels = []
    for row in df_bins.iterrows():  

        count = count + float(row[1]['count'])
        right = float(row[0].right)  
        calc_percent = (count / no_rows) * 100

        #print(count, calc_percent)
        if (calc_percent >= min_percent or step_value >= max_cum_step - 0.5):
            bins.append(right)
            labels.append('({},{}]'.format(left,right))
#             print("{}\t {}\t {}\t {}".format(left,right, count,round(calc_percent,2) ))        
            left = float(row[0].right)        
            count = 0
            step_value = 0
            continue

        step_value = step_value + step 
        
    bins.append((right+max_cum_step))
    labels.append('({},{}]'.format(left,right+max_cum_step))
#     print("{}\t {}\t {}\t {}".format(left,right, count,round(calc_percent,2) ))       
    return bins,labels


def categorise_column(x,categories_dict):
    no_of_good_features = 0
    no_of_bad_features = 0
    
    for col,value in x.items():
        if col in categories_dict and value>0:
            if value>=categories_dict[col]['good_feature_value'] : 
                no_of_good_features = no_of_good_features +1
            elif value<=categories_dict[col]['bad_feature_value'] : 
                no_of_bad_features = no_of_bad_features +1
                
    return no_of_good_features,no_of_bad_features

def categorise_features_to_columns(x,categories_dict):
    no_of_good_features = 0
    no_of_bad_features = 0
    
    no_of_good_features,no_of_bad_features = categorise_column(x,categories_dict)
    
    return pd.Series([no_of_good_features,no_of_bad_features])


def change_dict_path_value(dotted_path, org,value,delim='.'):
    paths, current = dotted_path.split(sep=delim), org
    for p in paths[:-1]:
        if is_number(p) : current = current[int(p)]
        else : current = current[p]
    current[paths[-1]] = value
    return org

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
    
def calculate_accuracy_and_acc_per_bucket(X_test,y_test,neural_network=True):
    if neural_network : yhat = model.model.predict(X_test)
    else : yhat = model.predict(X_test)
    yhat_argmax = np.argmax(yhat,axis=1) 
    ytest_argmax = np.argmax(y_test,axis=1)
    buckets = []
    bucket_dict = {}
    for x,y in zip(ytest_argmax,encoder.inverse_transform(ytest_argmax)):
        if (x,y) not in buckets:
            buckets.append((x,y))
    buckets.sort(key=lambda tup: tup[0])

    for bucket in buckets:
        bucket_dict[bucket[1]] = [0,0]

    count = 0
    for yi,yihat in zip(encoder.inverse_transform(ytest_argmax),encoder.inverse_transform(yhat_argmax)):
        if yi==yihat:
            bucket_dict[yi][0] = bucket_dict[yi][0] + 1
        else : 
            bucket_dict[yi][1] = bucket_dict[yi][1] + 1


    list_of_keys = []
    for key in list(bucket_dict.keys()):
        list_of_keys.append(key)
    accurracies_per_pricebucket = pd.DataFrame(bucket_dict,index=['correct_pred','wrong_pred'])
    
    accurracies_per_pricebucket = accurracies_per_pricebucket.transpose()
    accurracies_per_pricebucket['bucket_accuracy'] = accurracies_per_pricebucket.apply((lambda x : x.correct_pred/(x.correct_pred+x.wrong_pred)*100),axis=1)
    return accurracies_per_pricebucket

# Classes

In [ ]:
class Model():

    def __init__(self,configs,no_of_features,no_of_labels):
        self.configs = configs
        self.no_of_features = no_of_features
        self.layer_count = 1
        self.no_of_labels = no_of_labels
#         self.model = Sequential()

    def build_model(self):
        configs = self.configs
        no_of_features = self.no_of_features
        optimizer = configs['model']['optimizer']
        self.model = Sequential()
        for layer in configs['model']['layers']:
            neurons = layer['neurons'] if 'neurons' in layer else None
            dropout_rate = layer['rate'] if 'rate' in layer else None
            activation = layer['activation'] if 'activation' in layer else None
            return_seq = layer['return_seq'] if 'return_seq' in layer else None
            input_timesteps = layer['input_timesteps'] if 'input_timesteps' in layer else None
            input_dim = layer['input_dim'] if 'input_dim' in layer else None
            output_dim = layer['output_dim'] if 'output_dim' in layer else None
            
            if layer['type'] == 'dense':
                if input_dim:
                    self.model.add(Dense(neurons,input_dim=no_of_features, activation=activation))
                    print("Layer {} -> Dense input layer".format(self.layer_count))
                    self.layer_count= self.layer_count+1
                elif output_dim:
                    self.model.add(Dense(self.no_of_labels, activation=activation))
                    print("Layer {} -> Dense output layer".format(self.layer_count))
                else:
                    self.model.add(Dense(neurons, activation=activation))
                    if neurons==1 and activation: print("Layer {} -> Dense output layer. Activation -{}-".format(self.layer_count,activation))
                    else: print("Layer {} -> Dense hidden layer".format(self.layer_count))
                    self.layer_count= self.layer_count+1
            if layer['type'] == 'lstm':
                self.model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences=return_seq))
            if layer['type'] == 'dropout':
                self.model.add(Dropout(dropout_rate))
                print("Layer {} -> Dropout hidden layer".format(self.layer_count))
                self.layer_count= self.layer_count+1
        if optimizer == 'SGD':
            print("optimizer = SGD")
            optimizer = SGD(lr=0.01)
        else:
            print("Optimizer = {}".format(optimizer))
        print("Loss = {}".format(configs['model']['loss']))
        self.model.compile(loss=configs['model']['loss'], optimizer=optimizer,metrics=['mse','mae', 'accuracy'])
        
        print('[Model] Model Compiled')
        print(self.model.summary())
    
#     def kfold_fit_model(self,X,Y,configs,no_of_features,regression=True,):
#         epochs = configs['training']['epochs']
#         batch_size = configs['training']['batch_size']
#         seed = 7
#         np.random.seed(seed)
#         if regression:
#             estimator = KerasRegressor(build_fn=self.build_model,epochs=epochs,batch_size=batch_size,verbose=1)
#         else:
#             estimator = KerasClassifier(build_fn=self.build_model,epochs=epochs,batch_size=batch_size,verbose=1)
        
# #         estimators = []
# #         estimators.append(('standardize',MinMaxScaler(feature_range = (0, 1)) ))
# #         estimators.append(('mlp',estimator))
# #         pipeline = Pipeline(estimators)
        
#         kfold = KFold(n_splits=10, random_state=seed)
#         results = cross_val_score(estimator, X, Y, cv=kfold)
        
#         print("Kfold results : %.2f (%.2f) MSE" % (results.mean(), results.std()))
        
    def fit_model(self,X_train,y_train,configs,no_of_features):
        epochs = configs['training']['epochs']
        batch_size = configs['training']['batch_size']
        early_stop = EarlyStopping(monitor='accuracy',
                              min_delta=0,
                              patience=round(epochs*0.1),
                              verbose=0, mode='auto')
        history = self.model.fit(X_train,y_train,batch_size=batch_size,nb_epoch=epochs,callbacks=[early_stop])
        plt.figure(figsize=(20,10))
        plt.plot(history.history['mean_squared_error'])
        plt.figure(figsize=(20,10))
        plt.plot(history.history['mean_absolute_error'])
        plt.show()
        return history
    
    def predict_yhat(self,test_X):
        return self.model.predict(test_X)
        

In [ ]:
# from keras.wrappers.scikit_learn import KerasRegressor,KerasClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline

# class Model():

#     def __init__(self,):
#         self.model = Sequential()

#     def build_model(self, configs,no_of_features):

#         for layer in configs['model']['layers']:
#             neurons = layer['neurons'] if 'neurons' in layer else None
#             dropout_rate = layer['rate'] if 'rate' in layer else None
#             activation = layer['activation'] if 'activation' in layer else None
#             return_seq = layer['return_seq'] if 'return_seq' in layer else None
#             input_timesteps = layer['input_timesteps'] if 'input_timesteps' in layer else None
#             input_dim = layer['input_dim'] if 'input_dim' in layer else None

#             if layer['type'] == 'dense':
#                 print("Input dim = {}".format(input_dim))
#                 if input_dim:
#                     self.model.add(Dense(neurons,input_dim=no_of_features, activation=activation))
#                 else:
#                     self.model.add(Dense(neurons, activation=activation))
#             if layer['type'] == 'lstm':
#                 self.model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences=return_seq))
#             if layer['type'] == 'dropout':
#                 self.model.add(Dropout(dropout_rate))

#         #self.model.compile(loss=configs['model']['loss'], optimizer=configs['model']['optimizer'],metrics=['mse', 'mae', 'mape'])
        
#         print('[Model] Model Compiled')
        
#     def kfold_fit_model(self,X,Y,configs,no_of_features,regression=True,):
#         epochs = configs['training']['epochs']
#         batch_size = configs['training']['batch_size']
#         seed = 7
#         np.random.seed(seed)
#         if regression:
#             estimator = KerasRegressor(build_fn=self.build_model(configs,no_of_features),epochs=epochs,batch_size=batch_size,verbose=1)
#         else:
#             estimator = KerasClassifier(build_fn=self.build_model(configs,no_of_features),epochs=epochs,batch_size=batch_size,verbose=1)
        
#         estimators = []
#         estimators.append(('standardize',MinMaxScaler(feature_range = (0, 1)) ))
#         estimators.append(('mlp',estimator))
#         pipeline = Pipeline(estimators)
        
#         kfold = KFold(n_splits=10, random_state=seed)
#         results = cross_val_score(pipeline, X, Y, cv=kfold)
        
#         print("Kfold results : %.2f (%.2f) MSE" % (results.mean(), results.std()))
        
#     def fit_model(self,X_train,y_train,configs,regression=True):
#         epochs = configs['training']['epochs']
#         batch_size = configs['training']['batch_size']
#         early_stop = EarlyStopping(monitor='mean_squared_error',
#                               min_delta=0,
#                               patience=round(epochs*0.2),
#                               verbose=0, mode='auto')
#         history = self.model.fit(X_train,y_train,batch_size=batch_size,nb_epoch=epochs,callbacks=[early_stop])
#         plt.figure(figsize=(20,10))
#         plt.plot(history.history['mean_squared_error'])
#         plt.figure(figsize=(20,10))
#         plt.plot(history.history['mean_absolute_error'])
#         plt.figure(figsize=(20,10))
#         plt.plot(history.history['mean_absolute_percentage_error'])
#         plt.show()
    
#     def predict_yhat(self,test_X):
#         return self.model.predict(test_X)
        

In [ ]:
class DataLoader():
    """A class for loading and transforming data for the lstm model"""

    def __init__(self, filename,config):
        self.dataF = pd.read_csv(filename,delimiter='^').round(2)
        self.prepared_data = self.prepare_data(self.dataF,config)
        
    def prepare_data(self,df,config):
        
        max_no_of_features = config['data']['no_of_features']
        corr_threshold = config['data']['corr_threshold']
        fs_method = config['data']['fs_method']
        
        #remove unwanted columns and rows
        df = pre_process_df(df,verbose=0)
                
        #Remove outliers
        outlier_conf = config['data']['outliers']
        if outlier_conf['method']=='IQR':
            df = remove_outliers_IQR(df,high_quantile=outlier_conf['high_quantile'])
        elif outlier_conf['method']=='zscore':
            df = remove_outliers_zscore(df,threshold=outlier_conf['threshold'])
        elif outlier_conf['method']=='3_ex':
            df = df[df['price_amount'] // df['ex_conv_rate']<= 3]
        elif outlier_conf['method']=='brute':
            df = df[df['price_amount'] // df['ex_conv_rate']<= 3]
            df = df[df['price_amount'] >=0.5]
        elif outlier_conf['method']=='cut_low':
            low_pr_df = df.loc[(df['price_amount']<0.5) & (df['days_in_snapshot']>=3)]
            df.drop(low_pr_df.index,inplace=True)
        elif outlier_conf['method'] == 'quantile':
            quantile_price = df['price_amount'].quantile(0.99)
            df =df[df['price_amount']<= quantile_price]    
        
        #if a column has a high percentage of missing values make all values higher than 0(zero) equal to 1
        if config['data']['high_percentage_missing_values']:
            dataF = df.drop('price_amount',axis=1).copy()
            cols = dataF.columns
            count = 0
            for col in cols:
                column_zero_percentage = (dataF[col]!=0).sum()/(len(dataF[col]))*100
                if column_zero_percentage<5:
                    dataF[col] = dataF[col].apply(lambda x: 1 if x>0 else 0)
                    count= count+1
            dataF['price_amount'] = df['price_amount']
            df = dataF.copy()
        
        if config['data']['feature_selection']:
            
            #Feature selection prep
            important_features = feature_selection(df,method=fs_method, verbose=1,importance_threshold=corr_threshold,max_no_of_features=max_no_of_features)

            #Actual feature selection
            df = df[important_features]
            print("Shape before {}".format(df.shape))
            
            df['no_of_ex_features'] = (df.filter(regex='ex_.+')!=0).sum(axis=1)-1
            df = df[df['no_of_ex_features']>0]
            
            print("Shape after {}".format(df.shape))
            print('Feature Selection method works')
        
        
        feature_category_dict = {}
        with open('bad_good_features_categegorization.json','r') as fjson:
            feature_category_dict = json.load(fjson)
        
        df[['no_of_good_features','no_of_bad_features']] = df.apply(lambda x : categorise_features_to_columns(x,feature_category_dict),axis=1)
        
        
        
        self.no_features = df.shape[1]
        
        if config['data']['dynamic_price_bins']:
            df['price_amount'] = self.convert_to_price_range(df,configs)
        
        return df
    
    def rearrange_yhat_to_first_column(self,df,yhat_name='price_amount'):
    
        rearranged_columns = ['price_amount']
        for c in df.columns:
            if c=='price_amount' :continue
            rearranged_columns.append(c)
        return df[rearranged_columns]
    
    def split_X_Y(self):
        rearranged_df_values = self.rearrange_yhat_to_first_column(self.prepared_data).values
        X_data = rearranged_df_values[:,1:] 
        Y_data = rearranged_df_values[:,0]
        return X_data,Y_data
    
    def convert_to_price_range(self,dataset,configs):
        df = dataset.copy()
        df = df[['price_amount']]
        norm = 10
        mn = 0
        mx = int(round(df.max())) * norm
        step = 0.5
        step_adj = int(0.5 * norm)


        bins = pd.cut(df['price_amount'], [x/10 for x in range(mn,mx, step_adj)])
        df_bins = df.groupby(bins)['price_amount'].agg(['count'])
        no_rows = df_bins['count'].sum()

        df_bins['percent'] = round(df_bins  * 100 / no_rows, 2) 

        # --- FUNC params
        min_percent = 3
        no_rows = df_bins['count'].sum()
        max_cum_step = 50
        step = 0.5
        
        bins,labels = dynamic_bins(df_bins,no_rows,min_percent=configs['data']['min_percent'],max_cum_step=configs['data']['max_cum_step'])
        df['price_range'] = pd.cut(df['price_amount'],bins=bins,labels=labels,include_lowest=True)
        df.drop(['price_amount'],axis=1,inplace=True)
        df.rename(columns={'price_range':'price_amount'},inplace=True)
        self.no_of_labels = len(labels)
        return df

# Rare Analysis w/o Kfold and GridSearch

## Chests

### SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import seaborn as sns

configs = json.load(open('chest_class_config.json', 'r'))

data = DataLoader(configs['data']['filename'],configs)

#Trim data using "days_in_snapshot" feature
data.prepared_data = data.prepared_data[(data.prepared_data['days_in_snapshot']<=2) &(data.prepared_data['days_in_snapshot']>=0)]

no_of_classes = data.prepared_data['price_amount'].nunique()
X,y = data.split_X_Y()

no_of_features = X.shape[1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
svm = SVC(kernel='linear')

# 2. Fit
svm.fit(X_train, y_train)

# 3. Predict 
y_pred = svm.predict(X_test)
acc = accuracy_score(y_pred, y_test)
acc

### K- nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

configs = json.load(open('chest_class_config.json', 'r'))

data = DataLoader(configs['data']['filename'],configs)

#Trim data using "days_in_snapshot" feature
data.prepared_data = data.prepared_data[(data.prepared_data['days_in_snapshot']<=2) &(data.prepared_data['days_in_snapshot']>=0)]

no_of_classes = data.prepared_data['price_amount'].nunique()
X,y = data.split_X_Y()

no_of_features = X.shape[1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

knn = KNeighborsClassifier(weights='distance',n_neighbors=1000,leaf_size=100)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(weights='distance',n_neighbors=no_of_classes,leaf_size=100)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred))

### Clustering with KMeans

In [ ]:
from sklearn.cluster import KMeans

configs = json.load(open('chest_class_config.json', 'r'))

data = DataLoader(configs['data']['filename'],configs)

#Trim data using "days_in_snapshot" feature
data.prepared_data = data.prepared_data[(data.prepared_data['days_in_snapshot']<=2) &(data.prepared_data['days_in_snapshot']>=0)]

no_of_classes = data.prepared_data['price_amount'].nunique()
X,y = data.split_X_Y()

no_of_features = X.shape[1]

data.no_of_labels = full_data2.price_amount.nunique()

no_of_classes = data.no_of_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Instantiate
kmc = KMeans(n_clusters=no_of_classes, max_iter=1000, n_init=20)

# Fit
kmc.fit(X_train, y_train)
y_pred = kmc.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.cluster import KMeans


data.no_of_labels = full_data2.price_amount.nunique()

no_of_classes = data.no_of_labels

kmc = KMeans(n_clusters=no_of_classes, max_iter=1000, n_init=20)

# Fit
kmc.fit(X_train, y_train)
y_pred = kmc.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

### Decision Trees

In [ ]:
from sklearn import metrics
configs = json.load(open('chest_class_config.json', 'r'))

data = DataLoader(configs['data']['filename'],configs)

#Trim data using "days_in_snapshot" feature
data.prepared_data = data.prepared_data[(data.prepared_data['days_in_snapshot']<=2) &(data.prepared_data['days_in_snapshot']>=0)]

no_of_classes = data.prepared_data['price_amount'].nunique()
X,y = data.split_X_Y()

no_of_features = X.shape[1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# 1. Instantiate
# default criterion=gini
# you can swap to criterion=entropy 
dtc = DecisionTreeClassifier(random_state=0)

# 2. Fit
dtc.fit(X_train, y_train)

# 3. Predict, there're 4 features in the iris dataset
y_pred = dtc.predict(X_test)

metrics.accuracy_score(y_test, y_pred)

### XGBOOST with custom bins

In [ ]:
from sklearn.metrics import accuracy_score
from xgboost.sklearn import XGBClassifier
from keras import backend as K
K.clear_session()
# load data
configs = json.load(open('chest_class_config.json', 'r'))

data = DataLoader(configs['data']['filename'],configs)
full_data = data.prepared_data.copy()

#Trim data using "days_in_snapshot" feature

data.prepared_data = full_data[(full_data['days_in_snapshot']<=10) &(full_data['days_in_snapshot']>=0)]

data.no_of_labels = full_data.price_amount.nunique()

no_of_classes = data.no_of_labels
X,y = data.split_X_Y()
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)
sc = MinMaxScaler(feature_range=(0,1))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# fit model no training data

model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

### Deep Learning

In [ ]:
configs = json.load(open('chest_class_config.json', 'r'))

data = DataLoader(configs['data']['filename'],configs)

full_data = data.prepared_data.copy()

#Trim data using "days_in_snapshot" feature

data.prepared_data = full_data[(full_data['days_in_snapshot']<=10) &(full_data['days_in_snapshot']>=0)]

data.no_of_labels = full_data.price_amount.nunique()

In [ ]:
%%capture

all_accuracies_permutations = {}

configs = json.load(open('chest_class_config.json', 'r'))

grid = ParameterGrid({"training.epochs": [20,30,40,50],
                      "model.layers.5.activation":['softmax'],
                      "data.outliers.method":['nothing','IQR'],
                     })

for param_list in list(grid):
        
    for key,value in param_list.items():
        configs = change_dict_path_value(key,configs,value)
    
    if K.backend() == 'tensorflow':
        K.clear_session()

    data.prepared_data = full_data2[(full_data2['days_in_snapshot']<=10) &(full_data2['days_in_snapshot']>=0)]

    data.no_of_labels = full_data2.price_amount.nunique()

    no_of_classes = data.no_of_labels
    X,y = data.split_X_Y()

    encoder = LabelEncoder()
    encoder.fit(y)
    encoded_Y = encoder.transform(y)
    # convert integers to dummy variables (i.e. one hot encoded)
    y = np_utils.to_categorical(encoded_Y)

    no_of_features = X.shape[1]

    
    permutation_key = "epochs{}_outliers{}_features{}".format(configs['training']['epochs'],
                                                           configs['data']['outliers']['method'],
                                                           no_of_features)
    
    
    #configs = json.load(open('chest_class_config.json', 'r'))
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)
    sc = MinMaxScaler(feature_range=(0,1))
    #sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    print("X.shape[1] = {}".format(X.shape[1]))
    model = Model(configs,X.shape[1],no_of_classes)
    model.build_model()
    history = model.fit_model(X_train,y_train,configs,X.shape[1]) 

    
    accuracies_per_bucket = calculate_accuracy_and_acc_per_bucket(X_test,y_test)
    all_accuracies_permutations[permutation_key] = accuracies_per_bucket
    

In [ ]:
for key, value in all_accuracies_permutations.items():
    key
    value

In [ ]:
yhat = model.model.predict(X_test)
yhat_argmax = np.argmax(yhat,axis=1) 
ytest_argmax = np.argmax(y_test,axis=1)
metrics.accuracy_score(ytest_argmax,yhat_argmax)

# Tests

## Partial dependence calculator

In [ ]:
configs = json.load(open('chest_config.json', 'r'))

data = DataLoader(configs['data']['filename'],configs)

In [ ]:
full_data = data.prepared_data.copy()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble.partial_dependence import plot_partial_dependence
from sklearn.ensemble.partial_dependence import partial_dependence

X,y = data.split_X_Y()
no_of_features = X.shape[1]
names = full_data.drop('price_amount',axis=1).columns
print("Training GBRT...")
clf = GradientBoostingRegressor(n_estimators=100, max_depth=10,
                                learning_rate=0.1, loss='ls',
                                random_state=1)
clf.fit(X, y)
print(" done.")
features = range(0,no_of_features,1)


names = full_data.drop('price_amount',axis=1).columns[0]
features=[0]
for i in range(0,no_of_features):
    features = [i]
    names = full_data.drop('price_amount',axis=1).columns[i] +(' '*i)
    fig, axs = plot_partial_dependence(clf, X, features,
                                   feature_names=names,n_cols=10,
                                   n_jobs=5, grid_resolution=10)
    axs[0].set_xlabel(names)
    plt.show()


## price_range bucketing

In [ ]:
df[df['price_amount']<=0.5][df['days_in_snapshot']>=2]['price_amount'].count()

In [ ]:
out = pd.cut(df['price_amount'],bins=[0,0.5,10000])
out_norm = out.value_counts(sort=False, normalize=False)
# pyplot.figure(figsize=(40,20))
out_norm.plot.bar(rot=0, color="b", figsize=(30,15))
# ax.set_xticklabels([c[1:-1].replace(","," to") for c in out.cat.categories])
pyplot.ylabel("pct")
# pyplot.figure(figsize=(40,20))
out_norm
# df['price_distr']
# pyplot.bar(df['price_distr'],height=20)

In [ ]:
out = pd.cut(df['price_amount'],bins=[0,0.5,1,2,5,10,50,100,200,300,400,500,600,700,800,900,1000,10000])
out_norm = out.value_counts(sort=False, normalize=True).mul(100)
# pyplot.figure(figsize=(40,20))
out_norm.plot.bar(rot=0, color="b", figsize=(30,15))
# ax.set_xticklabels([c[1:-1].replace(","," to") for c in out.cat.categories])
pyplot.ylabel("pct")
# pyplot.figure(figsize=(40,20))
# df['price_distr']
# pyplot.bar(df['price_distr'],height=20)